In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# change file paths if necessary

train_x = np.load('/content/drive/MyDrive/ML/ML_data/bread_train_images.npy')
train_y = np.load('/content/drive/MyDrive/ML/ML_data/bread_train_labels.npy')

test_x = np.load('/content/drive/MyDrive/ML/ML_data/bread_test_images.npy')
test_y = np.load('/content/drive/MyDrive/ML/ML_data/bread_test_labels.npy')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ML/ML_data/bread_train_images.npy'

In [ ]:
print(train_x.shape)
print(train_y.shape)

In [ ]:
print(test_x.shape)
print(test_y.shape)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')# Google Drive를 Colab에 마운트해서 파일을 불러올 수 있게 함

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 데이터 로드

# 학습 이미지라벨 로드
train_x = np.load('/content/drive/MyDrive/ML/ML_data/bread_train_images.npy')
train_y = np.load('/content/drive/MyDrive/ML/ML_data/bread_train_labels.npy')

# 테스트 이미지라벨 로드
test_x = np.load('/content/drive/MyDrive/ML/ML_data/bread_test_images.npy')
test_y = np.load('/content/drive/MyDrive/ML/ML_data/bread_test_labels.npy')

# 로드된 데이터의 shape 확인
print("train_x shape:", train_x.shape)
print("train_y shape:", train_y.shape)
print("test_x shape :", test_x.shape)
print("test_y shape :", test_y.shape)

# 2) 데이터 전처리

# 이미지가 (N, H, W) 형태면 채널 차원(C=1)을 추가해서 (N, H, W, 1)로 만듦
if train_x.ndim == 3:
    train_x = train_x[..., np.newaxis]
if test_x.ndim == 3:
    test_x = test_x[..., np.newaxis]

# 정수형 이미지(0~255)인 경우 float32로 변환 후 0~1로 정규화
train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

# 만약 값 범위가 0~255로 보이면 정규화(0~1)
if train_x.max() > 1.0:
    train_x /= 255.0
if test_x.max() > 1.0:
    test_x /= 255.0

# 라벨 형태 정리
train_y = np.squeeze(train_y)
test_y = np.squeeze(test_y)

# 라벨이 one-hot인지 정수 클래스인지 확인해서 class 개수 결정
if train_y.ndim == 2:
    num_classes = train_y.shape[1]
    y_format = "one-hot"
else:
    num_classes = int(np.max(train_y)) + 1
    y_format = "int"

print("After preprocessing:")
print("train_x:", train_x.shape, train_x.dtype, "min/max:", train_x.min(), train_x.max())
print("train_y:", train_y.shape, "format:", y_format, "num_classes:", num_classes)

# 데이터 샘플 시각화

# 데이터 정상 로드 확인
plt.figure(figsize=(10, 4))
for i in range(8):
    plt.subplot(2, 4, i+1)
    img = train_x[i].squeeze()
    plt.imshow(img, cmap='gray')# 흑백으로 표시
    plt.title(f"y={train_y[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

# CNN 모델 설계

# 입력 이미지 크기(H,W,C)
input_shape = train_x.shape[1:]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),# 입력 레이어 지정

    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),# 특징 추출(필터 32개)
    tf.keras.layers.MaxPooling2D((2,2)),# 크기 줄이기

    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Flatten(),# 2D feature map -> 1D 벡터
    tf.keras.layers.Dense(128, activation='relu'),# fully-connected
    tf.keras.layers.Dropout(0.3),# 과적합 방지
    tf.keras.layers.Dense(num_classes, activation='softmax')# 클래스 개수만큼 출력
])

# 모델 컴파일(학습 준비)

# 라벨이 정수 클래스면 sparse_categorical_crossentropy
# 라벨이 one-hot이면 categorical_crossentropy
if y_format == "int":
    loss_fn = 'sparse_categorical_crossentropy'
else:
    loss_fn = 'categorical_crossentropy'

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),# Adam 최적화
    loss=loss_fn,# 라벨 형식에 맞는 loss
    metrics=['accuracy']# 정확도 출력
)

model.summary()# 모델 구조 출력

# 모델 학습

# 검증용으로 train의 일부를 validation_split로 떼어냄
history = model.fit(
    train_x, train_y,
    epochs=15,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# 7) 학습 곡선(accuracy/loss) 시각화

plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.legend(['train', 'val'])
plt.show()

plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'val'])
plt.show()

# 8) 테스트 성능 평가

test_loss, test_acc = model.evaluate(test_x, test_y, verbose=0)
print(f"Test loss: {test_loss:.4f}")
print(f"Test acc : {test_acc:.4f}")

# 9) 예측 + (가격 예측) 매핑

# 테스트 데이터 일부에 대해 예측 수행
pred_prob = model.predict(test_x[:16])
pred_cls = np.argmax(pred_prob, axis=1)

# 실제 라벨이 one-hot이면 정수 라벨로 변환해서 비교
if y_format == "one-hot":
    true_cls = np.argmax(test_y[:16], axis=1)
else:
    true_cls = test_y[:16]

print("Pred:", pred_cls)
print("True:", true_cls)

price_table = {
    0: 2000,
    1: 2500,
    2: 3000,
    3: 3500,
    4: 4000
}

# 클래스 개수가 더 많으면 자동으로 기본값(예: 0원) 넣도록 처리
def cls_to_price(c):
    return price_table.get(int(c), 0)

pred_price = [cls_to_price(c) for c in pred_cls]
true_price = [cls_to_price(c) for c in true_cls]

print("Pred price:", pred_price)
print("True price:", true_price)

# 예측 결과 시각화

plt.figure(figsize=(10, 5))
for i in range(16):
    plt.subplot(4, 4, i+1)
    img = test_x[i].squeeze()
    plt.imshow(img, cmap='gray')
    plt.title(f"T:{true_cls[i]} / P:{pred_cls[i]}\n₩{pred_price[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()
